In [1]:
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, ShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, confusion_matrix, make_scorer
from sklearn.svm import SVC
import pennylane.numpy as np
import pandas as pd
from utils import specificity_score, negative_prediction_value_score, gmean_score, informedness_score
from model_parameters import ModelParameters
import time

# Training SVM

In [2]:
model_parameters = ModelParameters()

# Initialize the model
model = SVC(kernel='linear', C=1.0) 

x_train, x_test, y_train, y_test = model_parameters.get_htru_2()

for i in range(model_parameters.num_runs):
    balanced_x_train, balance_y_train = model_parameters.sub_select_dataset(x_train, y_train, model_parameters.training_samples, balanced=True)
    balanced_x_test, balance_y_test = model_parameters.sub_select_dataset(x_test, y_test, model_parameters.testing_samples)

    # Train the model and make predictions
    start_training_time = time.time()
    model.fit(balanced_x_train, balance_y_train)
    end_training_time = time.time()

    training_duration = end_training_time - start_training_time

    start_testing_time = time.time()
    y_pred = model.predict(balanced_x_test)
    end_testing_time = time.time()

    test_duration = end_testing_time - start_testing_time

    # Store the scores
    model_parameters.append_score(balance_y_test, y_pred, training_duration, test_duration)

# Prediction

In [3]:
# Calculate mean and standard deviation for each metric and print the results
for metric, values in model_parameters.scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")

Accuracy: 0.964 ± 0.015
Balanced_accuracy: 0.948 ± 0.015
Recall: 0.927 ± 0.030
Specificity: 0.968 ± 0.016
Precision: 0.759 ± 0.083
Npv: 0.992 ± 0.004
Gmean: 0.947 ± 0.016
Informedness: 0.895 ± 0.031
Training_duration: 0.003 ± 0.001
Testing_duration: 0.001 ± 0.000
